In [ ]:
# GPU
!nvidia-smi

In [ ]:
import os
import pandas as pd

sample = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

if len(sample) == 2477:
    sample.to_csv('submission.csv', index=False)
    os._exit(00)

In [ ]:
import sys
sys.path.append('../input/timm126')
sys.path.append('../input/timm126/timm126')
sys.path.append('../input/timm413')
sys.path.append('../input/timm413/timm413')
from timm126 import timm as timm_126
from timm413 import timm_new as timm_413
print(timm_126.__version__)
print(timm_413.__version__)

# DICOM to PNG

In [ ]:
# offline gdcm
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut


def read_xray(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data


def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    im = Image.fromarray(array)
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

In [ ]:
#public image_ids
public = pd.read_csv('../input/covid-public-test/sample_submission.csv')
public['type'] = public['id'].apply(lambda x: x[-5:])
public_image = public[public['type'] == 'image'].reset_index(drop=True)
public_image['image_id'] = public_image['id'].apply(lambda x: x[:-6])
public_image_ids = public_image['image_id'].to_list()

In [ ]:
image_id = []
dim0 = []
dim1 = []
sz = 512
save_dir = f'/kaggle/tmp/test/'
os.makedirs(save_dir, exist_ok=True)

for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/test')):
    for file in filenames:
        if file[:-4] in public_image_ids:
            continue
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=sz)  
        im.save(os.path.join(save_dir, file.replace('dcm', 'png')))
        image_id.append(file.replace('.dcm', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])

In [ ]:
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1})
meta.head()

In [ ]:
# test image root path
# all images: root + image_id +.png
root = '/kaggle/tmp/test'

# Detection

In [ ]:
!pip install '/kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' --no-deps > /dev/null

sys.path.append("/kaggle/input/timm-efficientdet-pytorch-fixdiv")
sys.path.append("/kaggle/input/omegaconf")
sys.path.append("/kaggle/input/weightedboxesfusion")

from ensemble_boxes import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold

import math
import gc, os
from datetime import datetime
import time
import random
import cv2
from glob import glob
import pydicom
from tqdm import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure
from bokeh.plotting import figure as bokeh_figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs
from PIL import Image
from sklearn import preprocessing
from random import randint

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import torch.optim as optim

from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet

import torchvision
from torchvision import transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR

import warnings
warnings.filterwarnings("ignore")
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 2020
seed_everything(seed)
NFOLDS = 5

#ImageNet
mean = np.array([[[0.485, 0.456, 0.406]]])
std = np.array([[[0.229, 0.224, 0.225]]])

In [ ]:
public = pd.read_csv('../input/covid-public-test/sample_submission.csv')
test = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

#submit
if len(test) != 2477:
    test = pd.concat([test, public]).reset_index(drop=True).drop_duplicates(keep=False).reset_index(drop=True)
    
test['type'] = test['id'].apply(lambda x: x[-5:])
test_study = test[test['type'] == 'study'].reset_index(drop=True)
test_image = test[test['type'] == 'image'].reset_index(drop=True)
test_study['study_id'] = test_study['id'].apply(lambda x: x[:-6])
test_image['image_id'] = test_image['id'].apply(lambda x: x[:-6])
test_study = test_study.rename(columns={'PredictionString': 'PredictionString1'})
test_image = test_image.rename(columns={'PredictionString': 'PredictionString2'})
test_study_ = test_study.drop(['type', 'study_id'], axis=1)
tmp = pd.DataFrame(columns=test_study.columns)

for i in tqdm(range(len(test_study))):
    study_id = test_study.iloc[i, 3]
    img_paths = glob(f'../input/siim-covid19-detection/test/{study_id}/*/*')
    if len(img_paths) == 1:
        img_path = img_paths[0]
        image_id = img_path.split('/')[-1][:-4]
        test_study.loc[i, 'image_id'] = image_id
    else:
        add_df = pd.concat([test_study.iloc[i:i+1]]*(len(img_paths)-1)).reset_index(drop=True)
        for j in range(len(img_paths)):
            if j == 0:
                img_path = img_paths[j]
                image_id = img_path.split('/')[-1][:-4]
                test_study.loc[i, 'image_id'] = image_id
            else:
                img_path = img_paths[j]
                image_id = img_path.split('/')[-1][:-4]
                add_df.loc[j-1, 'image_id'] = image_id
        tmp = tmp.append(add_df).reset_index(drop=True)

In [ ]:
test = pd.concat([test_study, tmp]).sort_values('id').reset_index(drop=True)
test = test.merge(meta, on='image_id').merge(test_image[['PredictionString2', 'image_id']], on='image_id')
test.head()

In [ ]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=sz, width=sz, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)


def collate_fn(batch):
    return tuple(zip(*batch))


class COVIDDataset(Dataset):
    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df
        self.transforms = transforms

    def __getitem__(self, index):
        image_id = self.df.iloc[index, 4]
        image = cv2.imread(f'{root}/{image_id}.png').astype(np.float32)
        width = self.df.iloc[index, 6]
        height = self.df.iloc[index, 5]
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id, width, height

    def __len__(self):
        return self.df.shape[0]

In [ ]:
def load_net(checkpoint_path, model_name):
    config = get_efficientdet_config(model_name)
    net = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    del checkpoint
    gc.collect()
    net = DetBenchEval(net, config)
    net.eval()
    return net.cuda()


def make_predictions(images, score_threshold=0.001):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
            })
    return [predictions]


def run_wbf(predictions, image_index, image_size=512, iou_thr=0.5, skip_box_thr=0.001, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = non_maximum_weighted(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr, weights=weights)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels


def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append(f"opacity {j[0]} {j[1][0]} {j[1][1]} {j[1][2]} {j[1][3]}")
    return " ".join(pred_strings)

In [ ]:
dataset = COVIDDataset(
    df=test,
    transforms=get_valid_transforms()
)

data_loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
# EFD3 inference
efd3 = test.copy()
print('<<<<<<<<<<EFD3>>>>>>>>>>')
for i in range(NFOLDS):
    print(f'=====FOLD {i}=====')
    net = load_net(checkpoint_path=f'../input/efd3ver001-weight/fold{i}-best.bin', model_name='tf_efficientdet_d3')
    results = []
    for images, image_ids, widths, heights in tqdm(data_loader):
        predictions = make_predictions(images)
        for j, image in enumerate(images):
            boxes, scores, labels = run_wbf(predictions, image_index=j)
            boxes = boxes.astype(np.float32).clip(min=0, max=sz)
            boxes[:, 0] = boxes[:, 0] / sz 
            boxes[:, 1] = boxes[:, 1] / sz
            boxes[:, 2] = boxes[:, 2] / sz
            boxes[:, 3] = boxes[:, 3] / sz
            result = {'PredictionString': format_prediction_string(boxes, scores)}
            results.append(result)
    efd3[f'PredictionString_{i}'] = np.nan
    for k in tqdm(range(len(efd3))):
        efd3.loc[k, f'PredictionString_{i}'] = f" {results[k]['PredictionString']}"  
    del net, results
    gc.collect()
    
efd3['id'] = efd3['image_id'].apply(lambda x: x + '_image')
efd3.head()

In [ ]:
# EFD4 inference
efd4 = test.copy()
print('<<<<<<<<<<EFD4>>>>>>>>>>')
for i in range(NFOLDS):
    print(f'=====FOLD {i}=====')
    net = load_net(checkpoint_path=f'../input/efd4ver002-weight/fold{i}-best.bin', model_name='tf_efficientdet_d4')
    results = []
    for images, image_ids, widths, heights in tqdm(data_loader):
        predictions = make_predictions(images)
        for j, image in enumerate(images):
            boxes, scores, labels = run_wbf(predictions, image_index=j)
            boxes = boxes.astype(np.float32).clip(min=0, max=sz)
            boxes[:, 0] = boxes[:, 0] / sz 
            boxes[:, 1] = boxes[:, 1] / sz
            boxes[:, 2] = boxes[:, 2] / sz
            boxes[:, 3] = boxes[:, 3] / sz
            result = {'PredictionString': format_prediction_string(boxes, scores)}
            results.append(result)
    efd4[f'PredictionString_{i}'] = np.nan
    for k in tqdm(range(len(efd4))):
        efd4.loc[k, f'PredictionString_{i}'] = f" {results[k]['PredictionString']}"  
    del net, results
    gc.collect()
    
efd4['id'] = efd4['image_id'].apply(lambda x: x + '_image')
efd4.head()

In [ ]:
# EFD5 inference
efd5 = test.copy()
print('<<<<<<<<<<EFD5>>>>>>>>>>')
for i in range(NFOLDS):
    print(f'=====FOLD {i}=====')
    net = load_net(checkpoint_path=f'../input/efd5ver005-weight/fold{i}-best.bin', model_name='tf_efficientdet_d5')
    results = []
    for images, image_ids, widths, heights in tqdm(data_loader):
        predictions = make_predictions(images)
        for j, image in enumerate(images):
            boxes, scores, labels = run_wbf(predictions, image_index=j)
            boxes = boxes.astype(np.float32).clip(min=0, max=sz)
            boxes[:, 0] = boxes[:, 0] / sz 
            boxes[:, 1] = boxes[:, 1] / sz
            boxes[:, 2] = boxes[:, 2] / sz
            boxes[:, 3] = boxes[:, 3] / sz
            result = {'PredictionString': format_prediction_string(boxes, scores)}
            results.append(result)
    efd5[f'PredictionString_{i}'] = np.nan
    for k in tqdm(range(len(efd5))):
        efd5.loc[k, f'PredictionString_{i}'] = f" {results[k]['PredictionString']}"  
    del net, results
    gc.collect()
    
efd5['id'] = efd5['image_id'].apply(lambda x: x + '_image')
efd5.head()

In [ ]:
!pip install ../input/pytorch-160-with-torchvision-070/torch-1.6.0cu101-cp37-cp37m-linux_x86_64.whl
!pip install ../input/pytorch-160-with-torchvision-070/torchvision-0.7.0cu101-cp37-cp37m-linux_x86_64.whl

In [ ]:
public = pd.read_csv('../input/covid-public-test/sample_submission.csv')
test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

#submit
if len(test_df) != 2477:
    test_df = pd.concat([test_df, public]).reset_index(drop=True).drop_duplicates(keep=False).reset_index(drop=True)
    
test_df['type'] = test_df['id'].apply(lambda x: x[-5:])
test_df = test_df[test_df['type'] == 'image'].reset_index(drop=True)
meta['id'] = meta['image_id'] + '_image'
test_df = test_df.merge(meta, on='id', how='left')
test_df = test_df.drop(['type', 'image_id'], axis=1)
test_df['split'] = 'test'
test_df['fold'] = -1
efd3 = test_df[['id']].merge(efd3, on='id')
efd4 = test_df[['id']].merge(efd4, on='id')
efd5 = test_df[['id']].merge(efd5, on='id')

In [ ]:
test_df

In [ ]:
import shutil
dim = 512
NFOLDS = 5
test_dir = root
tmp = pd.DataFrame(columns=test_df.columns)
iou_th = [0.60, 0.61, 0.61, 0.60, 0.60]
conf_th = 0.001
sTH = 0.001
lTH = 0.362
weights1 = [[1.2, 1.2, 1, 0.7], [1, 4, 2, 0], [1.1, 1.1, 1, 0.5], [1.5, 1.5, 1, 0.7], [1.5, 1.5, 1, 0.9]]

for i in range(NFOLDS):
    print(f'==========FOLD{i}==========')
    labels_dict = {}
    scores_dict = {}
    boxes_dict = {}
    test_df_ = test_df.copy()
    test_df_['PredictionString'] = ''
    test_df_['fold'] = i
    
    #yolo inference
    weights_dir = f'/kaggle/input/yolov5ver005-weight/fold{i}_best.pt'
    shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5/', f'/kaggle/working/yolov5/fold{i}')
    os.chdir(f'/kaggle/working/yolov5/fold{i}')
    !python detect.py --weights $weights_dir --img 512 --conf 0.001 --iou 0.5 --source $test_dir --save-txt --save-conf --exist-ok
    
    for j in tqdm(range(len(test_df_))):
        image_id = test_df_.iloc[j, 0][:-6]
        labels_dict[j] = []
        scores_dict[j] = []
        boxes_dict[j] = []
        
        #yolo
        labels_list = []
        scores_list = []
        boxes_list = []
        try:
            f = open(f'/kaggle/working/yolov5/fold{i}/runs/detect/exp/labels/{image_id}.txt', 'r')
            string = f.read().split(' ')
            string.pop(0)
            bbox_num = len(string) // 5

            for k in range(bbox_num):
                label = 0
                xmid = float(string[0+5*k])
                ymid = float(string[1+5*k])
                w = float(string[2+5*k])
                h = float(string[3+5*k])
                xmin = xmid - w/2
                xmax = xmid + w/2
                ymin = ymid - h/2
                ymax = ymid + h/2
                score = float(string[4+5*k][:-3])
                labels_list.append(label)
                scores_list.append(score)
                boxes_list.append([xmin, ymin, xmax, ymax])
            labels_dict[j].append(labels_list)
            scores_dict[j].append(scores_list)
            boxes_dict[j].append(boxes_list)
        except:
            labels_dict[j].append([])
            scores_dict[j].append([])
            boxes_dict[j].append([])
            
        #EFD3
        labels_list = []
        scores_list = []
        boxes_list = []
        try:
            string = efd3.loc[j, f'PredictionString_{i}'].split(' ')
            string.pop(0)
            bbox_num = len(string) // 6
            for k in range(bbox_num):
                label = 0
                score = float(string[1+6*k])
                xmin = float(string[2+6*k])
                ymin = float(string[3+6*k])
                xmax = float(string[4+6*k])
                ymax = float(string[5+6*k])
                labels_list.append(label)
                scores_list.append(score)
                boxes_list.append([xmin, ymin, xmax, ymax])
            labels_dict[j].append(labels_list)
            scores_dict[j].append(scores_list)
            boxes_dict[j].append(boxes_list)
        except:
            labels_dict[j].append([])
            scores_dict[j].append([])
            boxes_dict[j].append([])
            
        #EFD4
        labels_list = []
        scores_list = []
        boxes_list = []
        try:
            string = efd4.loc[j, f'PredictionString_{i}'].split(' ')
            string.pop(0)
            bbox_num = len(string) // 6
            for k in range(bbox_num):
                label = 0
                score = float(string[1+6*k])
                xmin = float(string[2+6*k])
                ymin = float(string[3+6*k])
                xmax = float(string[4+6*k])
                ymax = float(string[5+6*k])
                labels_list.append(label)
                scores_list.append(score)
                boxes_list.append([xmin, ymin, xmax, ymax])
            labels_dict[j].append(labels_list)
            scores_dict[j].append(scores_list)
            boxes_dict[j].append(boxes_list)
        except:
            labels_dict[j].append([])
            scores_dict[j].append([])
            boxes_dict[j].append([])
            
        #EFD5
        labels_list = []
        scores_list = []
        boxes_list = []
        try:
            string = efd5.loc[j, f'PredictionString_{i}'].split(' ')
            string.pop(0)
            bbox_num = len(string) // 6
            for k in range(bbox_num):
                label = 0
                score = float(string[1+6*k])
                xmin = float(string[2+6*k])
                ymin = float(string[3+6*k])
                xmax = float(string[4+6*k])
                ymax = float(string[5+6*k])
                labels_list.append(label)
                scores_list.append(score)
                boxes_list.append([xmin, ymin, xmax, ymax])
            labels_dict[j].append(labels_list)
            scores_dict[j].append(scores_list)
            boxes_dict[j].append(boxes_list)
        except:
            labels_dict[j].append([])
            scores_dict[j].append([])
            boxes_dict[j].append([])
    
        #WBF per fold
        print('=== WBF per fold ===')
        width = test_df_.loc[j, 'dim1']
        height = test_df_.loc[j, 'dim0']
        boxes, scores, labels = weighted_boxes_fusion(
            boxes_dict[j], 
            scores_dict[j], 
            labels_dict[j],
            iou_thr=iou_th[i], 
            skip_box_thr=conf_th, 
            weights=weights1[i]
        )
        boxes[:, 0] = boxes[:, 0] * width
        boxes[:, 1] = boxes[:, 1] * height
        boxes[:, 2] = boxes[:, 2] * width
        boxes[:, 3] = boxes[:, 3] * height
    
        if len(boxes) == 0:  #BBox_num == 0
            continue
        else:
            box_num = len(boxes)
            for b in range(box_num):
                #small box removed
                if ((boxes[b][2] - boxes[b][0]) * (boxes[b][3] - boxes[b][1])) / (width * height) < sTH:
                    continue
                
                #large box removed
                if ((boxes[b][2] - boxes[b][0]) * (boxes[b][3] - boxes[b][1])) / (width * height) > lTH:
                    continue
                
                #edge box removed
                if ((boxes[b][2] + boxes[b][0])/2 < width*0.05) | ((boxes[b][2] + boxes[b][0])/2 > width*0.95) | ((boxes[b][3] + boxes[b][1])/2 < height*0.05) | ((boxes[b][3] + boxes[b][1])/2 > height*0.95):
                    continue
                
                #horizontal box removed
                if (boxes[b][2] - boxes[b][0]) > width*0.6:
                    continue
                    
                try:
                    test_df_.loc[j, 'PredictionString'] += f'opacity {scores[b]} {int(boxes[b][0])} {int(boxes[b][1])} {int(boxes[b][2])} {int(boxes[b][3])} '
                except:
                    continue
                    
    tmp = tmp.append(test_df_, ignore_index=True)
    del labels_dict, scores_dict, boxes_dict
    gc.collect()

In [ ]:
shutil.rmtree('/kaggle/working/yolov5/fold0')
shutil.rmtree('/kaggle/working/yolov5/fold1')
shutil.rmtree('/kaggle/working/yolov5/fold2')
shutil.rmtree('/kaggle/working/yolov5/fold3')
shutil.rmtree('/kaggle/working/yolov5/fold4')
os.chdir(f'/kaggle/working')

In [ ]:
tmp

In [ ]:
#tmp = pd.read_csv('../input/detection-final-sub-df/det_inf_per_fold.csv')
tmp.to_csv('/kaggle/working/det_inf_per_fold.csv', index=False)
det_inf_per_fold = tmp

In [ ]:
# WBF across folds
weights2 = [1.5, 1, 1, 1.5, 2]  # 0.5982 / 0.5684 / 0.5637 / 0.5835 / 0.6316
ens_th = 0.65
id_list = tmp.id.unique()
labels_dict = {}
scores_dict = {}
boxes_dict = {}
test_df['PredictionString'] = ''

print('=== WBF across folds ===')
for i in tqdm(range(len(id_list))):
    image_id = id_list[i]
    dfs = tmp[tmp['id'] == image_id].sort_values('fold').reset_index(drop=True)
    width = dfs['dim1'].mean()
    height = dfs['dim0'].mean()
    labels_dict[i] = []
    scores_dict[i] = []
    boxes_dict[i] = []
    
    for j in range(NFOLDS):
        labels_list = []
        scores_list = []
        boxes_list = []
        try:
            string = dfs[dfs['fold'] == j].iloc[0, 1].split(' ')
            bbox_num = len(string) // 6
            for k in range(bbox_num):
                label = 0
                score = float(string[1+6*k])
                xmin = float(string[2+6*k]) / width
                ymin = float(string[3+6*k]) / height
                xmax = float(string[4+6*k]) / width
                ymax = float(string[5+6*k]) / height
                labels_list.append(label)
                scores_list.append(score)
                boxes_list.append([xmin, ymin, xmax, ymax])
            labels_dict[i].append(labels_list)
            scores_dict[i].append(scores_list)
            boxes_dict[i].append(boxes_list)
        except:
            labels_dict[i].append([])
            scores_dict[i].append([])
            boxes_dict[i].append([])
            
    boxes, scores, labels = weighted_boxes_fusion(
        boxes_dict[i], 
        scores_dict[i], 
        labels_dict[i],
        iou_thr=ens_th, 
        skip_box_thr=conf_th, 
        weights=weights2
    )
  
    boxes[:, 0] = boxes[:, 0] * width
    boxes[:, 1] = boxes[:, 1] * height
    boxes[:, 2] = boxes[:, 2] * width
    boxes[:, 3] = boxes[:, 3] * height
    
    if len(boxes) == 0:  #BBox_num == 0
        continue
    else:
        box_num = len(boxes)
        for b in range(box_num):
            #small box removed
            if ((boxes[b][2] - boxes[b][0]) * (boxes[b][3] - boxes[b][1])) / (width * height) < sTH:
                continue
                
            #large box removed
            if ((boxes[b][2] - boxes[b][0]) * (boxes[b][3] - boxes[b][1])) / (width * height) > lTH:
                continue
                
            #edge box removed
            if ((boxes[b][2] + boxes[b][0])/2 < width*0.05) | ((boxes[b][2] + boxes[b][0])/2 > width*0.95) | ((boxes[b][3] + boxes[b][1])/2 < height*0.05) | ((boxes[b][3] + boxes[b][1])/2 > height*0.95):
                continue
                
            #horizontal box removed
            if (boxes[b][2] - boxes[b][0]) > width*0.6:
                continue
              
            try:
                index = test_df[test_df['id'] == image_id].index
                test_df.loc[index, 'PredictionString'] += f'opacity {scores[b]} {int(boxes[b][0])} {int(boxes[b][1])} {int(boxes[b][2])} {int(boxes[b][3])} '
            except:
                continue
                
del labels_dict, scores_dict, boxes_dict
gc.collect()

In [ ]:
test_df

In [ ]:
test_image = test_df.drop('fold', axis=1)
test_image.head()

In [ ]:
test_image.to_csv('public_image.csv', index=False)

# Classification

In [ ]:
public = pd.read_csv('../input/covid-public-test/sample_submission.csv')
test = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

#submit
if len(test) != 2477:
    test = pd.concat([test, public]).reset_index(drop=True).drop_duplicates(keep=False).reset_index(drop=True)

test['type'] = test['id'].apply(lambda x: x[-5:])
test_study = test[test['type'] == 'study'].reset_index(drop=True)
test_study['study_id'] = test_study['id'].apply(lambda x: x[:-6])
test_image = test_image[['id', 'PredictionString']]
test_image['image_id'] = test_image['id'].apply(lambda x: x[:-6])

tmp = pd.DataFrame(columns=test_study.columns)
for i in tqdm(range(len(test_study))):
    study_id = test_study.iloc[i, 3]
    img_paths = glob(f'../input/siim-covid19-detection/test/{study_id}/*/*')
    if len(img_paths) == 1:
        img_path = img_paths[0]
        image_id = img_path.split('/')[-1][:-4]
        test_study.loc[i, 'image_id'] = image_id
    else:
        add_df = pd.concat([test_study.iloc[i:i+1]]*(len(img_paths)-1)).reset_index(drop=True)
        for j in range(len(img_paths)):
            if j == 0:
                img_path = img_paths[j]
                image_id = img_path.split('/')[-1][:-4]
                test_study.loc[i, 'image_id'] = image_id
            else:
                img_path = img_paths[j]
                image_id = img_path.split('/')[-1][:-4]
                add_df.loc[j-1, 'image_id'] = image_id
        tmp = tmp.append(add_df).reset_index(drop=True)
        
test_study = pd.concat([test_study, tmp]).sort_values('id').reset_index(drop=True)
test = test_study.merge(test_image[['PredictionString', 'image_id']], on='image_id')

In [ ]:
test

In [ ]:
class COVIDDataset(Dataset):
    def __init__(self, root, df, transform=None):
        self.root = root
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.iloc[idx, 4]
        img = cv2.imread(f'{root}/{image_id}.png')
        
        if self.transform:  #input shape should be: (sz, sz, 3)
            img = self.transform(image=img)['image']
        
        img = (img/255.0 - mean) / std  #Normalization
        img = np.transpose(img, (2, 0, 1))  #shape(3, sz, sz)
        img = torch.from_numpy(img)

        return img

In [ ]:
def inference_fn1(data_loader, model, device):
    model.eval()    
    preds1 = []
    preds2 = []
    
    for i, x in tqdm(enumerate(data_loader)):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred1, pred2, pred3 = model(img)
            preds1.append(nn.Softmax()(pred1).detach().cpu().numpy())
            preds2.append(nn.Sigmoid()(pred2).detach().cpu().numpy())
            
        del img, pred1, pred2, pred3
        gc.collect()
        
    predictions1 = np.concatenate(preds1)
    predictions2 = np.concatenate(preds2).reshape(-1, 1)
    predictions = np.hstack([predictions1, predictions2])
    
    del preds1, preds2, predictions1, predictions2
    gc.collect()
    
    return predictions


def inference_fn2(data_loader, model, device):
    model.eval()    
    preds1 = []
    preds2 = []
    
    for i, x in tqdm(enumerate(data_loader)):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred1, pred2 = model(img)
            preds1.append(nn.Softmax()(pred1[:, :4]).detach().cpu().numpy())
            preds2.append(nn.Sigmoid()(pred1[:, 4]).detach().cpu().numpy())
            
        del img, pred1, pred2
        gc.collect()
        
    predictions1 = np.concatenate(preds1)
    predictions2 = np.concatenate(preds2).reshape(-1, 1)
    predictions = np.hstack([predictions1, predictions2])
    
    del preds1, preds2, predictions1, predictions2
    gc.collect()
    
    return predictions

In [ ]:
num_class1 = 4
num_class2 = 1

class Model0(nn.Module):
    def __init__(self):
        super(Model0, self).__init__()
        e = timm_413.create_model(
            'efficientnet_b3',
            pretrained=False, 
            drop_rate=0.3, 
            drop_path_rate=0.2
            )
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head,
            e.bn2,
            e.act2
        )
        self.logit1 = nn.Linear(1536, 4)
        self.logit2 = nn.Sequential(
            nn.Softmax(),
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )

        self.mask = nn.Sequential(
            nn.Conv2d(136, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0)
        )
        
    def forward(self, image):
        batch_size = len(image)
        x = 2 * image - 1
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        mask = self.mask(x)
        x = self.b6(x)
        x = self.b7(x)
        x = self.b8(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        logit1 = self.logit1(x)
        logit2 = self.logit2(logit1)
        return logit1, logit2, mask

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        #e = timm.create_model('tf_efficientnetv2_l', pretrained=True, in_chans=3)
        e = Model_Y()
        #e.load_state_dict(fix_key(torch.load("../data/efb3_chex_02_epoch{}_step{}.pth".format(6,18685))))
        e = e.model

        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head,
            e.bn2,
            e.act2
        )
        self.logit1 = nn.Linear(1536, num_class1)
        self.logit2 = nn.Sequential(
            nn.Softmax(),
            nn.Linear(num_class1, num_class1),
            nn.ReLU(),
            nn.Linear(num_class1, num_class2))
        self.mask = nn.Sequential(
            nn.Conv2d(136, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

    def forward(self, image):
        batch_size = len(image)
        x = 2 * image - 1
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        mask = self.mask(x)
        x = self.b6(x)
        x = self.b7(x)
        x = self.b8(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        logit1 = self.logit1(x)
        logit2 = self.logit2(logit1)
        return logit1, logit2, mask

class Model_Y(nn.Module):
    def __init__(self, out_size=14, pretrained=True):
        super().__init__()
        self.model = timm_413.create_model(
            'efficientnet_b3',
            pretrained=False, 
            drop_rate=0.3, 
            drop_path_rate=0.2
            )
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(1536, out_size)
    def forward(self, x):
        x = self.model(x)
        return x


class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        e = timm_413.create_model(
            'tf_efficientnetv2_l', 
            pretrained=False, 
            in_chans=3
        )
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head,
            e.bn2,
            e.act2
        )
        self.logit = nn.Linear(1280, 5)
        self.mask = nn.Sequential(
            nn.Conv2d(224, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )
        
    def forward(self, image):
        batch_size = len(image)
        x = 2 * image - 1
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        mask = self.mask(x)
        x = self.b6(x)
        x = self.b7(x)
        x = self.b8(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        logit = self.logit(x)
        
        return logit, mask
    
class Model_3(nn.Module):
    def __init__(self):
        super(Model_3, self).__init__()
        e = Model_X()
        e = e.model

        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head,
            e.bn2,
            e.act2
        )
        self.logit1 = nn.Linear(1280, 4)
        self.logit2 = nn.Sequential(
            nn.Softmax(),
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 1))
        self.mask = nn.Sequential(
            nn.Conv2d(224, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

    def forward(self, image):
        batch_size = len(image)
        x = 2 * image - 1
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        mask = self.mask(x)
        x = self.b6(x)
        x = self.b7(x)
        x = self.b8(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        logit1 = self.logit1(x)
        logit2 = self.logit2(logit1)
        return logit1, logit2, mask

class Model_X(nn.Module):
    def __init__(self, out_size=14, model_name="tf_efficientnetv2_l", pretrained=False):
        super().__init__()
        self.model = timm_413.create_model(model_name, pretrained=pretrained, in_chans=3)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(n_features, out_size),
        )
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
test['Negative'] = 0
test['Typical'] = 0
test['Indeterminate'] = 0
test['Atypical'] = 0
test['None'] = 0
target_cols = test.columns[6:].to_list()

In [ ]:
det_inf_per_fold = det_inf_per_fold.rename(columns={'id': 'image_id'})
det_inf_per_fold['image_id'] = det_inf_per_fold['image_id'].apply(lambda x: x[:-6])
det_inf_per_fold.head()

In [ ]:
test_ds = COVIDDataset(root=root, df=test, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=64, shuffle=False, num_workers=0)
NFOLDS = 5
all_predictions = 0
num_models = 7

for i in range(NFOLDS):
    print(f'==========FOLD{i}==========')
    predictions = 0
    
    model0 = Model0()
    model0.load_state_dict(torch.load(f'../input/covid-5class-efb3-seg-512-weight-0703-5/0703_5_fold_{i}.pth'))
    model0.to(device)
    predictions += inference_fn1(test_dl, model0, device) / (NFOLDS * num_models)
    del model0
    gc.collect()
    
    model1 = Model1()
    model1.load_state_dict(torch.load(f'../input/5cls-efb3-seg-512-chex-ep6-18685/model_1_fold_{i}.pth'))
    model1.to(device)
    predictions += inference_fn1(test_dl, model1, device) / (NFOLDS * num_models)
    del model1
    gc.collect()
    
    model2 = Model2()
    model2.load_state_dict(torch.load(f'../input/covid-5class-efv2l-seg-512-weight-0630-1/0630_1_fold_{i}.pth'))
    model2.to(device)
    predictions += inference_fn2(test_dl, model2, device) / (NFOLDS * num_models)
    del model2
    gc.collect()
    
    model3 = Model_3()
    model3.load_state_dict(torch.load(f'../input/5cls-efv2l-seg-512-chex-1ep/0630_1_fold_{i}.pth'))
    model3.to(device)
    predictions += inference_fn1(test_dl, model3, device) / (NFOLDS * num_models)
    del model3
    gc.collect()
    
    model4 = Model_3()
    model4.load_state_dict(torch.load(f'../input/5cls-efv2l-in21k-seg-512-chex-ep6-18685/model_1_fold_{i}.pth'))
    model4.to(device)
    predictions += inference_fn1(test_dl, model4, device) / (NFOLDS * num_models)
    del model4
    gc.collect()
    
    model5 = Model1()
    model5.load_state_dict(torch.load(f'../input/5cls-efb3-seg-focal-512-chex-ep6-18685/model_1_fold_{i}.pth'))
    model5.to(device)
    predictions += inference_fn1(test_dl, model5, device) / (NFOLDS * num_models)
    del model5
    gc.collect()
    
    from collections import OrderedDict
    def fix_key(state_dict):
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if k.startswith('module.'):
                k = k[7:]
            new_state_dict[k] = v
        return new_state_dict
    
    model6 = Model1()
    state_dict = torch.load(f'../input/5cls-efb3-seg-512-chex-ep6-18685-bs16/model_1_fold_{i}.pth')
    state_dict =  fix_key(state_dict)
    model6.load_state_dict(state_dict)
    model6.to(device)
    predictions += inference_fn1(test_dl, model6, device) / (NFOLDS * num_models)
    del model6
    gc.collect()
    
    #####################################################################
    #Post-Processing
    fold = det_inf_per_fold[det_inf_per_fold['fold'] == i].reset_index(drop=True)
    fold = test.iloc[:, 4:5].merge(fold, on='image_id')
    for j in tqdm(range(len(fold))):
        confs = []
        try:
            string = fold.loc[j, 'PredictionString'].split(' ')
            bbox_num = len(string) // 6
            for b in range(bbox_num):
                confs.append(float(string[6*b+1]))
            max_conf = max(confs)
            predictions[j, 0] *= (1 - max_conf)  #Negative
            predictions[j, 4] *= (1 - max_conf)  #None
        except:
            continue     
            
    all_predictions += predictions
    
test[target_cols] = all_predictions

In [ ]:
nn_mean = test['Negative'] * 0.55 + test['None'] * 0.45
test['Negative'] = nn_mean
test['None'] = nn_mean

w = 0.025
for i in tqdm(range(len(test))):
    opacity = 1 - test.loc[i, 'None']
    test.loc[i, 'Typical'] += opacity * w
    test.loc[i, 'Indeterminate'] += opacity * w
    test.loc[i, 'Atypical'] += opacity * w

In [ ]:
test.head(10)

In [ ]:
test_image = test[['image_id', 'None', 'PredictionString_y']]
test_image = test_image.rename(columns={'image_id': 'id', 'PredictionString_y': 'PredictionString'})
test_image['id'] = test_image['id'].apply(lambda x: x + '_image')

for i in tqdm(range(len(test_image))):
    none_conf = test_image.iloc[i, 1]
    test_image.iloc[i, 2] += f' none {none_conf} 0 0 1 1'
    
test_image = test_image.drop('None', axis=1)
test_image.head(10)

In [ ]:
test_study = test.drop(['type', 'study_id', 'image_id', 'PredictionString_x', 'PredictionString_y', 'None'], axis=1)
test_study = test_study.groupby('id').mean()
test_study['id'] = test_study.index
test_study = test_study.reset_index(drop=True).iloc[:, [4, 0, 1, 2, 3]]
test_study.head(10)

In [ ]:
labels = ["negative", "typical", "indeterminate", "atypical"]
dictionary = {"negative": 'Negative', "typical": 'Typical', "indeterminate": 'Indeterminate', "atypical": 'Atypical'}
test_study['PredictionString'] = np.nan

for i in tqdm(range(len(test_study))):
    s = ''
    for label in labels:
        s += f"{label} {test_study.loc[i, f'{dictionary[label]}']} 0 0 1 1" + ' '
    test_study.loc[i, 'PredictionString'] = s

In [ ]:
test_study = test_study[['id', 'PredictionString']]

In [ ]:
test = pd.concat([test_study, test_image]).reset_index(drop=True)

In [ ]:
test.head(20)

# Submission

In [ ]:
sample = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_dict = dict(zip(test['id'], test['PredictionString']))
sample['PredictionString'] = sample['id'].map(test_dict)
sample.to_csv('submission.csv', index=False)